# Fitting wavelength with multiple sample-detector distances

Wavelength and detector distance are correlated parameters when fitting the geometry using a single detector position. One should fix either one, or the other.
The simultaneous fitting of several images taken at various detector distances has proven to alleviate this limitation (https://doi.org/10.1107/S1600577519013328).

This tutorial explains how to perform a multi-position geometry refinement, this within pyFAI using a notebook. 

The dataset was recorded at the DanMAX beamline at MaxIV (Lund, Sweden) and made available by Mads Ry Jørgensen. It represents LaB6 reference material collected at 20keV at with a Pilatus detector at various distances from the sample.

## Loading data

All the data are stored into a single HDF5 file following the Nexus convention. This file has been reprocessed and differs from what is acquired at the DanMAX beamline.

In [ ]:
%matplotlib inline
#For documentation purpose, `inline` is used to enforce the storage of images into the notebook
# %matplotlib widget
import numpy
from matplotlib.pyplot import subplots
from pyFAI.goniometer import (GeometryTransformation, ExtendedTransformation,
                              GoniometerRefinement, Goniometer)
from pyFAI.calibrant import get_calibrant
from pyFAI.single_geometry import SingleGeometry
import h5py
import pyFAI
from pyFAI.gui import jupyter
import time
start_time = time.perf_counter()
import logging
logging.basicConfig(level=logging.WARNING)
print(f"Running pyFAI version {pyFAI.version}")

In [ ]:
#Download data from internet

from silx.resources import ExternalResources

#Comment out and configure the proxy if you are behind a firewall
#os.environ["http_proxy"] = "http://proxy.company.com:3128"

downloader = ExternalResources("pyFAI", "http://www.silx.org/pub/pyFAI/gonio/", "PYFAI_DATA")
data_file = downloader.getfile("LaB6_20keV.h5")

print("file downloaded:", data_file)

In [ ]:
h5 = h5py.File(data_file)
images = h5["entry_0000/DanMAX/Pilatus/data"][()]
distances = h5["entry_0000/DanMAX/sdd/value"][()]
energy = h5["entry_0000/DanMAX/monochromator/energy"][()]
print("Distances: ", distances)
print("Energy:", energy)
LaB6 = get_calibrant("LaB6")
wavelength_guess = pyFAI.units.hc/energy*1e-10
print("Wavelength:", wavelength_guess)
LaB6.wavelength = wavelength_guess

In [ ]:
fig, ax = subplots(2,3, figsize=(18,9))
for a,i in zip(ax.ravel(), images):
    jupyter.display(i, ax=a)

In [ ]:
detector = pyFAI.detector_factory("Pilatus2MCdTe")
detector.mask = numpy.min(images, axis=0)<0

This dataset is composed of 6 images collected between 150 and 650 mm with a Pilatus 2M CdTe detector. Debye-Scherrer rings are very nicely visible and a fully automated calibration will be performed.

## Automatic calibration

Since those images are pretty nice, one can read the beam-center position at (x=749, y=1573, ). The tilt and other distortion will be neglected in this first stage. We will now perform the automatic ring extraction

In [ ]:
geometries = {}
for dist, img  in zip(distances, images):
    ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(detector=detector, wavelength=wavelength_guess)
    ai.setFit2D(dist, 749, 1573)
    geo = SingleGeometry(dist, img, metadata=dist, calibrant=LaB6, detector=detector, geometry=ai)
    geometries[dist] = geo

In [ ]:
# Process the last image, the one with fewer rings: 

# First extract some control points:
geo.control_points  = geo.extract_cp()
# Visualization
ax = jupyter.display(sg=geo)

In [ ]:
print("Optimization of the geometry ... residual error is:", geo.geometry_refinement.refine2())
print(geo.geometry_refinement)

In [ ]:
# Extraction of the control points for all geometries:
fig, ax = subplots(2,3, figsize=(15, 10))
for a, lbl in zip(ax.ravel(), geometries):
    geo = geometries[lbl]
    geo.control_points  = geo.extract_cp()
#     print(geo.control_points)
    print(f"Optimization of the geometry {lbl}, residual error is: {geo.geometry_refinement.refine2()}")
    jupyter.display(sg=geo, ax=a)
a.get_legend().remove()

At this stage, we have 6 images and between 38 and 6 rings per image which is enough to start calibrating them all-together.

## Sample stage setup

We will optimize the energy in addition to all other parameters except the rotation along the beam (rot3).

* `dist0` represents the offset of the sample-detector stage. The associated `scale0` is expected to be 1e-3 to convert milimeters in meters.
* `poni1` represents the vertical position of the center and the associated `scale1` is expected to be null.
* `poni2` represents the horizontal position of the center and the associated `scale2` is expected to be null.
* All other rotation are expected to be null as well.

In [ ]:
goniotrans = ExtendedTransformation(param_names = ["dist0", "scale0", 
                                                   "poni1", "scale1",
                                                   "poni2", "scale2",
                                                   "rot1", "rot2", 
                                                   "energy"],
                                    dist_expr="dist0  + pos*scale0",
                                    poni1_expr="poni1 + pos*scale1",
                                    poni2_expr="poni2 + pos*scale2",
                                    rot1_expr="rot1",
                                    rot2_expr="rot2",
                                    rot3_expr="0",
                                    wavelength_expr="hc/energy*1e-10")


In [ ]:
# Starting parameters ...

param = {"dist0":   0.0,
         "poni1":  geo.geometry_refinement.poni1,
         "poni2":  geo.geometry_refinement.poni2,
         "rot1":   0.0,
         "rot2":   0.0,
         "scale0": 0.001,
         "scale1": 0.0,
         "scale2": 0.0,
         "energy": energy,
        }

In [ ]:
#Defines the bounds for some variables
bounds = {"dist0":  ( -0.1, 0.1),
          "poni1":  ( 0.0, 0.4),
          "poni2":  ( 0.0, 0.3),
          "rot1":   (-1.0, 1.0),
          "rot2":   (-1.0, 1.0),
          "scale0": (-1.1, 1.1),
          "scale1": (-1.1, 1.1),
          "scale2": (-1.1, 1.1),
          "energy": (energy-1,energy+1)
         }

In [ ]:
def distance(param):
    """Since the label is directly the distance ..."""
    return float(param)
assert 152.0 == distance(152)

In [ ]:
gonioref = GoniometerRefinement(param,         # Initial guess
                                bounds=bounds, # Enforces constrains
                                pos_function=distance,
                                trans_function=goniotrans,
                                detector=detector, 
                                wavelength=wavelength_guess)
print("Empty refinement object:", gonioref)
for lbl, geo in geometries.items():
    sg = gonioref.new_geometry(str(lbl), image=geo.image, metadata=lbl, control_points=geo.control_points, calibrant=LaB6)
    print(lbl, sg.get_position())
print("Populated refinement object:", gonioref)

At this stage, the GoniometerRefinement is fully populated and can directly be optimzied:

 ## Optimization of all parameters (including the energy)
 
All optimizer available in scipy are exposed in pyFAI, the default one is `slsqp` which takes into account bounds and other constrains. It is very robust but not the most precise. This is why we finish with a `simplex` step (without bounds).

In [ ]:
print("Global cost after refinement:", gonioref.refine3())

In [ ]:
# The `simplex` provides a refinement without bonds but more percise
gonioref.refine3(method="simplex")

In [ ]:
gonioref.save("table.json")

In [ ]:
print(open("table.json").read())

In [ ]:
gonioref.wavelength, gonioref._wavelength, 1e-10*pyFAI.units.hc/gonioref.nt_param(*gonioref.param).energy

In [ ]:
print("Ensure calibrant's wavelength has been updated:")
LaB6

In [ ]:
print(f"Total execution time: {time.perf_counter()-start_time:.3f}s")

This method ensures  the wavelength has been updated in all objects important for the calibration.

At this stage, calibration has been performed on a set of points extracted automatically. Each individual image should be controled to ensure control points are homogeneously distributed along the ring. If not, those images should see their control-points re-extracted (starting from the latest/best model) and refined again.
Basically, this comes down to running cells 8 and after again.

## Conclusions

* When the dataset is "clean", auto-extraction of control points works out of the box 
* Multi-position calibration to be performed in a minute once the model is known
* Energy can be refined with this methodology.